## **Inverse Kinematics Implementation**

In [1]:
import numpy as np
import math

In [2]:
L1 = 5.0 #coxa
L2 = 10.0 #femur
L3 = 15.0 #tibia

*We will calculate joint angles for the 3-DOF hexapod leg so as to position the foot at the target position coordiante  x y z which returns three angles in degree*

In [6]:
def inverse_kinematics(x, y, z):
  distance = np.sqrt(x**2 + y**2 + z**2)   #Total distance from origin
  maxima = L1 + L2 + L3
  #for checking if point is reachable
  if distance > maxima:
    return None
  minima = max(0, L1 - L2 - L3)
  if distance < minima:
    return None
  #calculating coxa angle alpha
  alpha =  math.atan2(y, x)

  x_prime = math.sqrt(x**2 + y**2) - L1  # it is the  distance from coxa joint to projection of end point
  h = z #its the height from base palne
  #now to calculate the distance from femur tibia joint to traget point
  d = math.sqrt(x_prime**2 + h**2)
  if d > L2 + L3:
   return None  #it becomes unreachable by femur and tibia

  #calculating femur angle beta and ensuring it lies within domain
  cos_beta_part = (L2**2 + d**2 - L3**2) / (2 * L2 * d)
  cos_beta_part = max(min(cos_beta_part, 1.0), -1.0)
  beta_part = math.acos(cos_beta_part)
  beta_offset = math.atan2(h, x_prime)
  beta = beta_part + beta_offset
  #here i calculated the angle between the femur and the ground plane + angle between the ground plane and the line to the target

  #calculating tibia angle gamma
  cos_gamma = (L2**2 + L3**2 - d**2) / (2 * L2 * L3)
  cos_gamma = max(min(cos_gamma, 1.0), -1.0)
  gamma = math.acos(cos_gamma)

  #converting from radians to degree to 2 decimal place
  alpha_deg = round(math.degrees(alpha), 2)
  beta_deg = round(math.degrees(beta), 2)
  gamma_deg = round(math.degrees(gamma), 2)

  return (alpha_deg, beta_deg, gamma_deg)

## **Testing**

In [7]:
def test_ik():
  test_cases = [
    (15, 10, 5),  #1.point withhin reachable area
    (5, 3, 2),    #2.point close to base
    (28, 0 , 0),  #3.point near the maximum reach of the leg
    (40, 0, 0),   #4. unreachable point as it exceeds L1+L2+L3
    (10, 10, -15) #5. point with large negative z
  ]
  print("HEXAPOD LEG INVERSE KINEMATICS TEST RESULTS")
  print("-" * 50)
  for i, (x, y, z) in enumerate(test_cases, 1):
    print(f"Test {i}:")
    print(f"Input target coordinates: (x={x}, y={y}, z={z})")
    result = inverse_kinematics(x, y, z)

    distance = np.sqrt(x**2 + y**2 + z**2)
    max_reach = L1 + L2 + L3
    if result is None:
      print(f"Result: UNREACHABLE (Distance {distance:.2f} exceeds maximum reach {max_reach})")
    else:
      alpha, beta, gamma = result
      print(f"Output joint angles: α={alpha}°, β={beta}°, γ={gamma}°")
      print(f"Result: REACHABLE")
    print("-" * 50)
if __name__ == "__main__":
  test_ik()


HEXAPOD LEG INVERSE KINEMATICS TEST RESULTS
--------------------------------------------------
Test 1:
Input target coordinates: (x=15, y=10, z=5)
Output joint angles: α=33.69°, β=96.53°, γ=64.26°
Result: REACHABLE
--------------------------------------------------
Test 2:
Input target coordinates: (x=5, y=3, z=2)
Output joint angles: α=30.96°, β=247.44°, γ=0.0°
Result: REACHABLE
--------------------------------------------------
Test 3:
Input target coordinates: (x=28, y=0, z=0)
Output joint angles: α=0.0°, β=28.57°, γ=132.84°
Result: REACHABLE
--------------------------------------------------
Test 4:
Input target coordinates: (x=40, y=0, z=0)
Result: UNREACHABLE (Distance 40.00 exceeds maximum reach 30.0)
--------------------------------------------------
Test 5:
Input target coordinates: (x=10, y=10, z=-15)
Output joint angles: α=45.0°, β=-0.14°, γ=86.86°
Result: REACHABLE
--------------------------------------------------
